# CMS search for a Z' boson in the single-lepton channel with full Run 2 dataset

This notebook demonstrates the Z' → tt̄ single-lepton analysis workflow on various AFs, including skimming, analysis, histogramming, and statistics steps.

## Workflow Overview

1. Setup Python path for intccms package
2. Install dependencies and register modules for cloud pickle
3. Acquire Dask client from AF environment
4. Configure analysis parameters
5. Run metadata extraction (`coffea` preprocessing)
6. Initialize analysis processor
7. Run processor with coffea.processor.Runner
8. Save histograms and run statistical analysis (if enabled)

## AF flag
We might want to run this code on different facilities, which may each have their own limitations or require different dask client setups. To make it easy to switch between facilities, just set the `AF` variable to the one of your choice. If your `AF` does not exist yet, you can introduce it in this notebook in the relevant sections.

In [1]:
AF="coffeacasa-condor" # options currently supported: [coffeacasa-condor, coffeacasa-gateway, purdue-af]
AUTO_CLOSE_CLIENT=False # the client setup is done with a contextmanager -- this flag decides if we automatically close the client as we exit the manager. If False, you handle closing manually. 

## Imports and dependencies

### The intccms package
The CMS implementation of the integration challenge is set in a package-like structure, which means we hae to add the source code to the python path. The package is referred to as `intccms`.

In [2]:
# Setup Python path to include intccms package
import sys
from pathlib import Path

# Add src directory to Python path
repo_root = Path.cwd()
src_dir = repo_root / "src"
examples_dir = repo_root
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))
if str(examples_dir) not in sys.path:
    sys.path.insert(0, str(examples_dir))
print(f"✅ Added {src_dir} to Python path")
print(f"✅ Added {examples_dir} to Python path")

✅ Added /home/cms-jovyan/intc/integration-challenge/cms/src to Python path
✅ Added /home/cms-jovyan/intc/integration-challenge/cms to Python path


### Installig extra dependencies
The `intccms` package requires `omegaconf` and `roastcoffea`, which is not by default on an AF. `roastcoffea` is a tool developed while working on this project and it provides an API to extract metrics from coffea-processor workflows. 

In [3]:
try:
    import omegaconf
except ImportError:
    print("⚠️ omegaconf not found, installing...")
    ! pip install omegaconf;

try:
    import roastcoffea
except ImportError:
    print("⚠️ roastcoffea not found, installing...")
    ! pip install roastcoffea;

### Alternative coffea version
In some cases, we might need to install our own `coffea` version which is not on the AF. For example, when testing a new feature or using a recently realased version with a fix.

In [4]:
COFFEA_VERSION = "2025.12.0"
COFFEA_PIP = f"coffea=={COFFEA_VERSION}" if "git" not in COFFEA_VERSION else COFFEA_VERSION

! pip install $COFFEA_PIP ;

# Pip-installable dependencies to install on workers
WORKER_DEPENDENCIES = [COFFEA_PIP, "roastcoffea==0.1.2"]

### Imports from stdlib and other libraries

In this notebook we use `dask` and `coffea`. 

In [5]:
# stdlib
import cloudpickle
import copy
import os
import time

from coffea.processor import DaskExecutor, IterativeExecutor
from coffea.nanoevents import NanoAODSchema

### Imports from intccms and other integration-challenge specific tooling

In [6]:
# intccms
from intccms.schema import Config, load_config_with_restricted_cli
from intccms.utils.output import OutputDirectoryManager
from intccms.metadata_extractor import DatasetMetadataManager
from intccms.datasets import DatasetManager
from intccms.analysis import run_processor_workflow

### Registering packages with cloudpickle
The intccms cannot be installed on the workers via `pip`, and the configuration files are in python modules which also cannot be installed on the workers. So we need to register them with `cloudpickle` to allow dask to serialize them and send them out.

In [7]:
import intccms
import example_cms

# Register modules for cloud pickle
cloudpickle.register_pickle_by_value(intccms)
cloudpickle.register_pickle_by_value(example_cms)

## Dask client setup

This notebook uses the `DaskExecutor` from `coffea` to distribute the task graph on the AF. The client setup varies in different facilities, so we implement a function which returns the correct client. The function does so by providing a context manager, within which the client is alive.

In [8]:
from intccms.utils.dask_client import acquire_client

## Configuration Setup

The CMS analysis implementation is configurable via python modules, which we have to import. For this notebook, the configuration files are found in `example_cms/configs/`. You can modify the modules in this directory manually, or you can dynamically change settings using python dictionary manipulation. Below are some settings of interest that you might want to tune when you are testing your setup.

In [9]:
# intccms configuration import
from example_cms.configs.configuration import config as original_config

# Create a deepcopy that we can manipulate
config = copy.deepcopy(original_config)

# Limit files for testing
config["datasets"]["max_files"] = 5 #None # None would run over all availale files

# Use local output directory
config["general"]["output_dir"] = "example_cms/outputs/"

# Preprocessing (coffea) can be executed once and results loaded
config["general"]["run_metadata_generation"] = True # If True, run analysis pre-processing

# Processer = Skimming (filter and save) + Analysis
config["general"]["run_processor"] = True  # If True, the coffea processor is executed
config["general"]["run_analysis"] = True # If True, the analysis part of the processor is executed
config["general"]["save_skimmed_output"] = False  # If True, skimmed events are saved to disk, otherwise filter executed on-the-fly

# Analysis = Systematics + histogramming + statsitics
config["general"]["run_histogramming"] = True
config["general"]["run_systematics"] = True
config["general"]["run_statistics"] = False

# Datasets to process, by default this is all datasets
#config["general"]["processes"] = ["data"] 

cli_args = [] # the code can be ran from CLI, but we don't care here
full_config = load_config_with_restricted_cli(config, cli_args)

# Validated config gives us a dictionary object with all settings checked to be safe with pydantic
validated_config = Config(**full_config)

## Running the Workflow

Running the CMS integration challenge workflow is split into a few steps, with a modular design that allows us flexibility. The steps are:

1. Setting up output directories
2. Building an input dataset manager
3. Running or loading the coffea preprocessing
4. Run the coffea processor

### Output manager setup

In [10]:
output_manager = OutputDirectoryManager(
    root_output_dir=validated_config.general.output_dir,
    cache_dir=validated_config.general.cache_dir,
    metadata_dir=validated_config.general.metadata_dir,
    skimmed_dir=validated_config.general.skimmed_dir
)

10:53:39 INFO     Output directory manager initialized with root:                                ]8;id=885006;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/utils/output/directories.py\directories.py]8;;\:]8;id=467412;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/utils/output/directories.py#169\169]8;;\
                  /home/cms-jovyan/intc/integration-challenge/cms/example_cms/outputs                              

## Configure Data Redirector (Optional)

Override the redirector in the config for accessing dataset files. Useful for testing different storage backends. You can also change this in `example_cms/configs/skim.py`

In [11]:
# Override redirector for all datasets
# Examples:
#   "root://xcache/"                    
#   "root://cmsxrootd.fnal.gov/"
#   "root://cms-xrd-global.cern.ch/"
REDIRECTOR = "root://xcache/"  # Change this to use a different redirector

print(f"Initial redirector  {validated_config.datasets.datasets[0].name}: {validated_config.datasets.datasets[0].redirector}")

# Apply to all datasets in config
for dataset in validated_config.datasets.datasets:
 dataset.redirector = REDIRECTOR

print(f"Redirector set to: {REDIRECTOR}")

# Verify the change
print(f"New redirector:  {validated_config.datasets.datasets[0].name}: {validated_config.datasets.datasets[0].redirector}")

Initial redirector  signal: root://xcache/
Redirector set to: root://xcache/
New redirector:  signal: root://xcache/


### Input dataset manager setup

In [12]:
dataset_manager = DatasetManager(validated_config.datasets)

         INFO     Initialized dataset manager with 10 datasets                                        ]8;id=442047;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/datasets/manager.py\manager.py]8;;\:]8;id=871118;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/datasets/manager.py#34\34]8;;\

### Coffea preprocessing

In [13]:
metadata_generator = DatasetMetadataManager(
  dataset_manager=dataset_manager,
  output_manager=output_manager,
  config=validated_config,
)

if metadata_generator.generate_metadata:
  with acquire_client(AF, close_after=AUTO_CLOSE_CLIENT, pip_packages=WORKER_DEPENDENCIES) as (client, cluster):
      metadata_generator.run(executor=DaskExecutor(client=client))
else:
  metadata_generator.run()  # No client needed

# Build metadata lookup and extract workitems
metadata_lookup = metadata_generator.build_metadata_lookup()
workitems = metadata_generator.workitems
;

         INFO     Initialized DatasetMetadataManager with output dir:                                ]8;id=350869;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/manager.py\manager.py]8;;\:]8;id=173888;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/manager.py#130\130]8;;\
                  /home/cms-jovyan/intc/integration-challenge/cms/example_cms/outputs/metadata                     

10:54:09 INFO     Connected to Dask scheduler                                                    ]8;id=228938;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/utils/dask_client.py\dask_client.py]8;;\:]8;id=308553;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/utils/dask_client.py#233\233]8;;\

         INFO     Dashboard: /user/mohamed.aly@cern.ch/proxy/8787/status                         ]8;id=24137;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/utils/dask_client.py\dask_client.py]8;;\:]8;id=221672;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/utils/dask_client.py#234\234]8;;\

         INFO     Starting metadata generation workflow...                                           ]8;id=480877;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/manager.py\manager.py]8;;\:]8;id=548476;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/manager.py#205\205]8;;\

         INFO     Building fileset for process: signal                                              ]8;id=751519;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/builders.py\builders.py]8;;\:]8;id=877019;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/builders.py#123\123]8;;\

...

         INFO     Saved JSON to                                                                           ]8;id=154558;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/io.py\io.py]8;;\:]8;id=865258;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/io.py#107\107]8;;\
                  /home/cms-jovyan/intc/integration-challenge/cms/example_cms/outputs/metadata/nanoaods_d          
                  ata_12_nominal.json                                                                              

         INFO     Saved JSON to                                                                           ]8;id=134043;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/io.py\io.py]8;;\:]8;id=149544;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/io.py#107\107]8;;\
                  /home/cms-jovyan/intc/integration-challenge/cms/example_cms/outputs/metadata/nanoaods_d          
                  ata_13_nominal.json                                                                              

         INFO     Metadata generation complete.                                                      ]8;id=466006;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/manager.py\manager.py]8;;\:]8;id=665474;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/manager.py#226\226]8;;\

         INFO     Built metadata lookup for 125 fileset keys                                         ]8;id=561046;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/manager.py\manager.py]8;;\:]8;id=112126;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/manager.py#443\443]8;;\

''

metadata_generator = DatasetMetadataManager(
  dataset_manager=dataset_manager,
  output_manager=output_manager,
  config=validated_config,
)

if metadata_generator.generate_metadata:
  with acquire_client(AF, close_after=AUTO_CLOSE_CLIENT, pip_packages=WORKER_DEPENDENCIES) as (client, cluster):
      metadata_generator.run(executor=DaskExecutor(client=client))
else:
  metadata_generator.run()  # No client needed

# Build metadata lookup and extract workitems
metadata_lookup = metadata_generator.build_metadata_lookup()
workitems = metadata_generator.workitems
;

In [14]:
# Run processor workflow
from intccms.analysis.processor import UnifiedProcessor

with acquire_client(AF, close_after=AUTO_CLOSE_CLIENT, pip_packages=WORKER_DEPENDENCIES) as (client, cluster):
    # Create processor instance for MetricsCollector
    unified_processor = UnifiedProcessor(
        config=validated_config,
        output_manager=output_manager,
        metadata_lookup=metadata_lookup,
    )

    t0 = time.perf_counter()
    output, report = run_processor_workflow(
        config=validated_config,
        output_manager=output_manager,
        metadata_lookup=metadata_lookup,
        workitems=workitems,
        executor=DaskExecutor(client=client, treereduction=8, retries=0),
        schema=NanoAODSchema,
    )
    t1 = time.perf_counter()


print(f"Processor workflow complete in {t1-t0:.1f} seconds!")

# Print summary
print(f"Total events processed: {output.get('processed_events', 0):,}")
print(f"Events after skim: {output.get('skimmed_events', 0):,}")
;

10:55:42 INFO     Connected to Dask scheduler                                                    ]8;id=501225;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/utils/dask_client.py\dask_client.py]8;;\:]8;id=686259;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/utils/dask_client.py#233\233]8;;\

         INFO     Dashboard: /user/mohamed.aly@cern.ch/proxy/8787/status                         ]8;id=216557;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/utils/dask_client.py\dask_client.py]8;;\:]8;id=158303;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/utils/dask_client.py#234\234]8;;\

         INFO     Initialized UnifiedProcessor: save_skimmed_output=False, analysis=True,          ]8;id=171378;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/analysis/processor.py\processor.py]8;;\:]8;id=683559;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/analysis/processor.py#195\195]8;;\
                  histogramming=True, systematics=True                                                             

         INFO     Running processor over data...                                                      ]8;id=352208;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/analysis/runner.py\runner.py]8;;\:]8;id=177904;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/analysis/runner.py#109\109]8;;\

         INFO     Initialized UnifiedProcessor: save_skimmed_output=False, analysis=True,          ]8;id=798796;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/analysis/processor.py\processor.py]8;;\:]8;id=340008;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/analysis/processor.py#195\195]8;;\
                  histogramming=True, systematics=True                                                             

...

         INFO     Histograms written to ROOT file:                                                ]8;id=578539;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/utils/output/histograms.py\histograms.py]8;;\:]8;id=703179;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/utils/output/histograms.py#137\137]8;;\
                  /home/cms-jovyan/intc/integration-challenge/cms/example_cms/outputs/histograms/                  
                  histograms.root                                                                                  

         INFO     Saved processor histograms (ROOT) to                                             ]8;id=406815;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/analysis/processor.py\processor.py]8;;\:]8;id=320999;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/analysis/processor.py#405\405]8;;\
                  /home/cms-jovyan/intc/integration-challenge/cms/example_cms/outputs/histograms/h                 
                  istograms.root                                                                                   

         INFO     Processor complete: 6,827,277,103 events processed, 297,458,382 events after skim   ]8;id=311336;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/analysis/runner.py\runner.py]8;;\:]8;id=998735;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/analysis/runner.py#205\205]8;;\

Processor workflow complete in 875.6 seconds!
Total events processed: 6,827,277,103
Events after skim: 297,458,382


''